In [4]:
import torch 
import torch.nn as nn
from torch.optim import SGD,lr_scheduler
from torch.nn import Sequential, Linear, Embedding
from torch.utils.data import DataLoader
import torchtext




##other library
import numpy as np
import datasets
import os 
import tqdm
import collections



In [5]:
device=('cuda' if torch.cuda.is_available() else 'cpu')
print('device: ',device)

device:  cuda


In [6]:
seed=1234
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic=True


In [7]:
train_data,test_data=datasets.load_dataset('imdb',split=['train','test'])

In [8]:
train_data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [10]:
train_data['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [11]:
train_data.features ## have two labels neg and pos

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [12]:
print(train_data[0]['text'][:100])
print(train_data[0]['label'])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it w
0


In [13]:
train_data[0].keys()

dict_keys(['text', 'label'])

Tokenization

ML models trains on numerical value only. So we will convert each string to int value using tokenizer. In order to do so first we need to separate `str` to individual tokens, and laters converts them into number. 

- using,`tokenizer` we will convert strings into number of tokens first. 
- later, we will use look_up_table  to convert each tokens(in str) to int. We do this by creating a vocabulary.

In [15]:
tokenizer=torchtext.data.utils.get_tokenizer('basic_english')

In [16]:
sample_text='Hello, I am from Nepal!'

In [17]:
sample_tokenized_text=tokenizer(sample_text)
print(sample_tokenized_text)

['hello', ',', 'i', 'am', 'from', 'nepal', '!']


In [18]:
## sometimes the str might contain to many unnecessary tokens; so we will take max_length and define by function

def tokenize_example(example,tokenizer,max_length):
    tokens=tokenizer(example['text'])[:max_length]
    return {'tokens':tokens}

## we return dictionary coz train_data are in the same format.  When use with map fucntion later we will see new tokens (with key) is appended in train_data format

In [19]:
train_data[0]['text'][:100] ##we will convert this to tokens using the above fucntion

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it w'

In [20]:
token_dict_sample=tokenize_example(train_data[0],tokenizer,100)

In [21]:
len(token_dict_sample['tokens'])

100

In [22]:
max_length=256

train_data=train_data.map(tokenize_example,
                          fn_kwargs={'tokenizer':tokenizer,
                                     'max_length':max_length})

test_data=test_data.map(tokenize_example,
                          fn_kwargs={'tokenizer':tokenizer,
                                     'max_length':max_length})

In [23]:
train_data.features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None),
 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [24]:
train_data

Dataset({
    features: ['text', 'label', 'tokens'],
    num_rows: 25000
})

In [25]:
len(train_data[0]['tokens']) ### mapped from above function 

256

In [26]:
len(train_data['tokens']) ## will load will athe tokens

25000

In [27]:
train_data['tokens'][0]

['i',
 'rented',
 'i',
 'am',
 'curious-yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 '.',
 'i',
 'also',
 'heard',
 'that',
 'at',
 'first',
 'it',
 'was',
 'seized',
 'by',
 'u',
 '.',
 's',
 '.',
 'customs',
 'if',
 'it',
 'ever',
 'tried',
 'to',
 'enter',
 'this',
 'country',
 ',',
 'therefore',
 'being',
 'a',
 'fan',
 'of',
 'films',
 'considered',
 'controversial',
 'i',
 'really',
 'had',
 'to',
 'see',
 'this',
 'for',
 'myself',
 '.',
 'the',
 'plot',
 'is',
 'centered',
 'around',
 'a',
 'young',
 'swedish',
 'drama',
 'student',
 'named',
 'lena',
 'who',
 'wants',
 'to',
 'learn',
 'everything',
 'she',
 'can',
 'about',
 'life',
 '.',
 'in',
 'particular',
 'she',
 'wants',
 'to',
 'focus',
 'her',
 'attentions',
 'to',
 'making',
 'some',
 'sort',
 'of',
 'documentary',
 'on',
 'what',
 'the',
 'average',
 'swede',
 'thought',


In [28]:
train_data[0]['tokens']==train_data['tokens'][0]
## however, train_data['tokens'][0] will load all dataset, and select first index. So, prefer to use train_data[0]['tokens'] to be used

True

In [29]:
train_data[0]['tokens'][:25]

['i',
 'rented',
 'i',
 'am',
 'curious-yellow',
 'from',
 'my',
 'video',
 'store',
 'because',
 'of',
 'all',
 'the',
 'controversy',
 'that',
 'surrounded',
 'it',
 'when',
 'it',
 'was',
 'first',
 'released',
 'in',
 '1967',
 '.']

In [30]:
## now we will split 25% of train_data to val_data

train_valid_data=train_data.train_test_split(test_size=0.25)
train_data=train_valid_data['train']
val_data=train_valid_data['test']

In [31]:
len(train_data) ## total data

18750

## Creating a vocabulary



In [34]:
min_freq=1
special_tokens=['unk','pad'] ## vocabs which appears less than 5 times or not present will be give 'unk' tokens, and if sample have 

example_tokens=[['Hi', 'I', 'am', "Akash"],
                ['Hi', "how","am","are"],
                ['ez','sir']]

example_vocab=torchtext.vocab.build_vocab_from_iterator(
    example_tokens,
    min_freq=min_freq,
    specials=special_tokens
)

In [35]:
## see the how vocab is used
example_vocab.get_itos() ## first vocabs is filled with special tokens, later the most repeated tokens first. If same freq is same then according to alphabetical value 

['unk', 'pad', 'Hi', 'am', 'Akash', 'I', 'are', 'ez', 'how', 'sir']

In [36]:
## Check the IDX value 
print('unk idx :',example_vocab['unk'])
print('Akash: ',example_vocab['Akash'])


unk idx : 0
Akash:  4


In [37]:
## checking if the word is in vocabulary
print("Akash" in example_vocab)
print("akash" in example_vocab)


True
False


In [38]:
## What if the tokens is not in dictionary
try:
    example_vocab['akash']
except:
    print("Can't access the tokens in vocabs ") 

## so in order to return 'unk' tokens or 0 value for tokens not in dictionary we have to do thus
unk_idx=example_vocab['unk']
example_vocab.set_default_index(unk_idx)

Can't access the tokens in vocabs 


In [42]:
example_vocab['akash'] ## return 0 for unknown tokens

0

In [43]:
##list of tokens to indices
example_tokens_to_indices=example_vocab.lookup_indices(["Hi", "Nepal"])
print(example_tokens_to_indices)

[2, 0]


Creating Vocabulary on train_data

Note: We only create vocabs on train_data; Not on test data so that information is not leaked. 

In [46]:
min_freq=5
special_tokens=['unk','pad'] ## vocabs which appears less than 5 times or not present will be give 'unk' tokens, and if sample have 

vocab=torchtext.vocab.build_vocab_from_iterator(
    train_data['tokens'],
    min_freq=min_freq,
    specials=special_tokens
)

In [47]:
print('sample tokenized example: ',sample_tokenized_text)
# print('ids from tokens: ',vocab(sample_tokenized_text)) ## converted to int value

sample tokenized example:  ['hello', ',', 'i', 'am', 'from', 'nepal', '!']


In [48]:
vocab.set_default_index(0)

In [49]:
vocab['hello']

4826

In [50]:
vocab.lookup_indices(sample_tokenized_text)

[4826, 4, 12, 219, 44, 20667, 36]

In [51]:
vocab.lookup_indices(train_data[0]['tokens'])[:25]

[1162,
 4,
 54,
 46,
 4,
 46,
 213,
 35,
 5,
 1031,
 7,
 2,
 37,
 65,
 223,
 1000,
 116,
 171,
 99,
 3,
 34,
 9,
 16,
 15,
 31]

In [52]:
## we will add the indices features to train_data

def  numericialize_tokens(example,vocab):
    ids=vocab.lookup_indices(example['tokens'])
    return {'ids':ids}

train_data=train_data.map(numericialize_tokens,
                          fn_kwargs={'vocab':vocab})


val_data=val_data.map(numericialize_tokens,
                          fn_kwargs={'vocab':vocab})

test_data=test_data.map(numericialize_tokens,
                        fn_kwargs={'vocab':vocab})

Map:   0%|          | 0/18750 [00:00<?, ? examples/s]

Map: 100%|██████████| 25000/25000 [00:08<00:00, 2916.71 examples/s]


In [53]:
print('mapped train_data.features: ',train_data.features)

mapped train_data.features:  {'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['neg', 'pos'], id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}


In [54]:
train_data[0]['ids'][:25]

[1162,
 4,
 54,
 46,
 4,
 46,
 213,
 35,
 5,
 1031,
 7,
 2,
 37,
 65,
 223,
 1000,
 116,
 171,
 99,
 3,
 34,
 9,
 16,
 15,
 31]

In [55]:
## creatomg tp pytoprch
train_data=train_data.with_format(type='torch',columns=['ids','label']) ## convert "ids" and "label" features to torch format ; Not  other
test_data=test_data.with_format(type='torch',columns=['ids','label'])
val_data=val_data.with_format(type='torch',columns=['ids','label'])

In [57]:
type(train_data[0]['ids'])

torch.Tensor

In [59]:
def get_collate_fnc(pad_index):
    
    def collate_fnc(batch):
        batch_ids=[i['ids'] for i in batch]
        batch_ids=torch.nn.utils.rnn.pad_sequence(batch_ids,padding_value=pad_index,batch_first=True)
        batch_label=[i['label'] for i in batch]
        batch_label=torch.stack(batch_label)
        batch={'ids':batch_ids,'label':batch_label}
        return batch
    return collate_fnc


In [60]:
## define Dataloader

def get_data_loader(dataset,batch_size,pad_index,shuffle=False):
    collate_fnc=get_collate_fnc(pad_index=pad_index)
    data_loader=DataLoader(dataset,
                           batch_size=batch_size,
                           collate_fn=collate_fnc,
                           shuffle=shuffle)
    return data_loader

In [61]:
## set shuffle ==True only for train_dataloader
batch_size=512
pad_idx=0
train_data_loader=get_data_loader(train_data,batch_size=batch_size,pad_index=pad_idx,shuffle=True)
test_data_loader=get_data_loader(test_data,batch_size=batch_size,pad_index=pad_idx)
val_data_loader=get_data_loader(val_data,batch_size=batch_size,pad_index=pad_idx)

In [62]:
for num,batch in enumerate(train_data_loader):
    if num==10:
        break

    print(batch['ids'].shape)

train_data_loader=get_data_loader(train_data,batch_size=batch_size,pad_index=pad_idx,shuffle=True)
    

torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])
torch.Size([512, 256])


In [48]:
batch['ids'].shape ## batch_size,max_length

torch.Size([512, 256])

In [75]:
### defining a Neural Bag of Words
class NBoW(nn.Module):

    def __init__(self,vocab_size,embedding_dim,output_dim):
        super(NBoW,self).__init__()
        self.embedding=nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim) ## return batch_size,num_tokens,emdbedding_dim
        self.fc=nn.Linear(in_features=embedding_dim,out_features=output_dim)

    def forward(self,x):
        embedding_output=self.embedding(x) ## return batch_size,num_tokens,emdbedding_dim
        polled_output=embedding_output.mean(dim=1) ## returns batch_size,embedding_dim
        prediction=self.fc(polled_output)
        return prediction

In [77]:
## defining the training parameters 

vocab_size=len(vocab)
embedding_dim=300
output_dim=len(train_data.unique('label'))

In [78]:
##testing the model on defined model

nbow=NBoW(vocab_size,embedding_dim,output_dim).to(device=device)
ids=batch['ids'].to(device=device)
print('ids.shape :',ids.shape)

test_output=nbow(ids)
print('test_output.shape: ',test_output.shape)
print('real_label.shape')

ids.shape : torch.Size([512, 256])
test_output.shape:  torch.Size([512, 2])
real_label.shape


In [80]:
a=torch.tensor([1,2,3,4,5],dtype=torch.float32)
b=torch.tensor([1,2,3,8,9],dtype=torch.float32)


In [83]:
a.eq(b).sum()/2

tensor(1.5000)

In [84]:
## defining accuracy

def get_accuracy(prediction,label):
    batch_size,_=prediction.shape
    prediction_classes=prediction.argmax(dim=1)
    corrected_prediction=prediction_classes.eq(label).sum()
    accuracy=corrected_prediction/batch_size
    return accuracy ## return tnesor as well

In [85]:
def training(data_loader,device,model,criterion,optimizer):
    model.train()
    epoch_losses=[]
    epoch_accuracy=[]

    for batch in tqdm.tqdm(data_loader,desc='...........training...........'):
        ids=batch['ids'].to(device=device)
        label=batch['label'].to(device=device)
        prediction=model(ids)
        loss=criterion(prediction,label)
        accuracy=get_accuracy(prediction,label)
        optimizer.zero_grad()
        optimizer.step()
        epoch_losses.append(loss.item())
        epoch_accuracy.append(accuracy.item())

    return np.mean(epoch_losses),np.mean(epoch_accuracy)



In [86]:
def evaluate(data_loader,device,model,criterion,optimizer):
    model.eval()
    epoch_losses=[]
    epoch_accuracy=[]

    for batch in tqdm.tqdm(data_loader,desc='...........Evaluating...........'):
        ids=batch['ids'].to(device=device)
        label=batch['label'].to(device=device)
        prediction=model(ids)
        loss=criterion(prediction,label)
        accuracy=get_accuracy(prediction,label)
        optimizer.zero_grad()
        optimizer.step()
        epoch_losses.append(loss.item())
        epoch_accuracy.append(accuracy.item())

    return np.mean(epoch_losses),np.mean(epoch_accuracy)

In [87]:
##defining criterion
criterion=nn.CrossEntropyLoss().to(device=device)
optimizer=SGD(nbow.parameters(),lr=0.001)

In [88]:
save_dire="../saved_model/nbow.pt"
n_epochs=100
best_valid_loss=float("inf")

metrics=collections.defaultdict(list)

for epoch in range(n_epochs):
    train_loss,train_acc=training(train_data_loader,device=device,
                                  model=nbow,criterion=criterion,
                                  optimizer=optimizer)
    
    val_loss,val_acc=evaluate(val_data_loader,device=device,
                                  model=nbow,criterion=criterion,
                                  optimizer=optimizer)
    
    metrics['train_loss'].append(train_loss)
    metrics['train_acc'].append(train_acc)
    metrics['val_loss'].append(val_loss)
    metrics['val_acc'].append(val_acc)
    if val_loss<train_loss:
        print('OKOKOK')
        best_valid_loss=val_loss
        
        torch.save(nbow.state_dict(),save_dire)

    print("Epoch Num: ", epoch)
    print('Train Loss: {:.3f}   | Train Acc: {:.3f}'.format(train_loss,train_acc))
    print('Val Loss: {:.3f}   | Val Acc: {:.3f}'.format(val_loss,val_acc))




## best Validation loss

nbow.load_state_dict(torch.load(save_dire)).to(device=device)
best_val_loss,best_val_acc=evaluate(val_data_loader,device=device,
                                  model=nbow,criterion=criterion,
                                  optimizer=optimizer)

print('Best Val Loss: {:.3f}   | Best Val Acc: {:.3f}'.format(best_val_loss,best_val_acc))



    

...........Evaluating...........: 100%|██████████| 13/13 [00:00<00:00, 25.69it/s]


Epoch Num:  0
Train Loss: 0.700   | Train Acc: 0.502
Val Loss: 0.700   | Val Acc: 0.497


...........Evaluating...........: 100%|██████████| 13/13 [00:00<00:00, 36.60it/s]


OKOKOK
Epoch Num:  1
Train Loss: 0.700   | Train Acc: 0.502
Val Loss: 0.700   | Val Acc: 0.497


...........Evaluating...........: 100%|██████████| 13/13 [00:00<00:00, 36.61it/s]


...

...........Evaluating...........: 100%|██████████| 13/13 [00:00<00:00, 37.00it/s]


Epoch Num:  98
Train Loss: 0.700   | Train Acc: 0.502
Val Loss: 0.700   | Val Acc: 0.497


...........Evaluating...........: 100%|██████████| 13/13 [00:00<00:00, 24.67it/s]


OKOKOK
Epoch Num:  99
Train Loss: 0.700   | Train Acc: 0.502
Val Loss: 0.700   | Val Acc: 0.497


AttributeError: '_IncompatibleKeys' object has no attribute 'to'

In [57]:
import os 


In [60]:
os.listdir('../saved_model/')

[]